## MLZoomcamp: Homework 5

In [24]:
import pickle
import requests

# Multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Question 1
* Install Pipenv
* What's the version of pipenv you installed?
* Use --version to find out

In [3]:
!pipenv --version

pipenv, version 2021.5.29


In [4]:
!pipenv install scikit-learn==1.0 flask

We recommend setting this in ~/.profile (or equivalent) for proper expected behavior.
Creating a virtualenv for this project...
Pipfile: /Users/rmcmaster/Documents/git_repos/pub_portfolio/mlz/05-deployment/Pipfile
Using /Users/rmcmaster/opt/anaconda3/envs/ml-zoomcamp/bin/python3 (3.8.11) to create virtualenv...
⠙ Creating virtual environment...created virtual environment CPython3.8.11.final.0-64 in 1453ms
  creator CPython3Posix(dest=/Users/rmcmaster/.local/share/virtualenvs/05-deployment-nvbrQS9s, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/Users/rmcmaster/Library/Application Support/virtualenv)
    added seed packages: pip==21.2.4, setuptools==58.1.0, wheel==0.37.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator

✔ Successfully created virtual environment! 
Virtualenv location: /Users/rmcmaster/.local/share/virtual

### Question 2
* Use Pipenv to install Scikit-Learn version 1.0
* What's the first hash for scikit-learn you get in Pipfile.lock?

"sha256:121f78d6564000dc5e968394f45aac87981fcaaf2be40cfcd8f07b2baa1e1829",

### Models

We've prepared a dictionary vectorizer and a model.

They were trained (roughly) using this code:

features = ['tenure', 'monthlycharges', 'contract']
dicts = df[features].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X = dv.fit_transform(dicts)

model = LogisticRegression().fit(X, y)
Note: You don't need to train the model. This code is just for your reference.

And then saved with Pickle. Download them:

### Question 3

Let's use these models!

* Write a script for loading these models with pickle
* Score this customer:

In [16]:
dv_file = 'dv.bin'
model_file = 'model1.bin'

with open(dv_file, 'rb') as f_in:
    dv = pickle.load(f_in)

with open(model_file, 'rb') as f_in:
    model = pickle.load(f_in)

dv, model

/Users/rmcmaster/opt/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/rmcmaster/opt/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


(DictVectorizer(sparse=False), LogisticRegression())

In [13]:
customer = {"contract": "two_year", "tenure": 12, "monthlycharges": 19.7}

In [15]:
X = dv.transform(customer)
model.predict_proba(X)[0, 1]

0.11549580587832914

The churn probability for this customer is therefore 0.11549580587832914.

### Question 4

Now let's serve this model as a web service

* Install Flask and Gunicorn (or waitress, if you're on Windows)
* Write Flask code for serving the model
* Now score this customer using requests:
    * url = "YOUR_URL"
    * customer = {"contract": "two_year", "tenure": 1, "monthlycharges": 10}
    * requests.post(url, json=customer).json()
* What's the probability that this customer is churning?

In [19]:
!pipenv install gunicorn, numpy

Installing gunicorn,...
Traceback (most recent call last):
  File "/Users/rmcmaster/opt/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/requirements.py", line 98, in __init__
    req = REQUIREMENT.parseString(requirement_string)
  File "/Users/rmcmaster/opt/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/pkg_resources/_vendor/pyparsing.py", line 1654, in parseString
    raise exc
  File "/Users/rmcmaster/opt/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/pkg_resources/_vendor/pyparsing.py", line 1644, in parseString
    loc, tokens = self._parse( instring, 0 )
  File "/Users/rmcmaster/opt/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/pkg_resources/_vendor/pyparsing.py", line 1402, in _parseNoCache
    loc,tokens = self.parseImpl( instring, preloc, doActions )
  File "/Users/rmcmaster/opt/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/pkg_resources/_vendor/pyparsing.py", line 3417, in parseImpl
    loc, exprtokens = 

In [21]:
url = 'http://localhost:9696/predict'

customer2 = {"contract": "two_year", "tenure": 1, "monthlycharges": 10}
requests.post(url, json=customer2).json()

In [25]:
# Checking with previous customer
requests.post(url, json=customer).json()
# And vice versa
X = dv.transform(customer2)
model.predict_proba(X)[0, 1]

{'churn': False, 'churn_probability': 0.11549580587832914}

0.9988892771007961

### Docker

Install Docker. We will use it for the next two questions.

For these questions, I prepared a base image: agrigorev/zoomcamp-model:3.8.12-slim. You'll need to use it (see Question 5 for an example).

This image is based on python:3.8.12-slim and has a logistic regression model (a different one) as well a dictionary vectorizer inside.

This is how the Dockerfile for this image looks like:

FROM python:3.8.12-slim
WORKDIR /app
COPY ["model2.bin", "dv.bin", "./"]
I already built it and then pushed it to agrigorev/zoomcamp-model:3.8.12-slim.

Note: You don't need to build this docker image, it's just for your reference.

### Question 5

Now create your own Dockerfile based on the image I prepared.

It should start like that:

FROM agrigorev/zoomcamp-model:3.8.12-slim
\# add your stuff here
Now complete it:

* Install all the dependencies form the Pipenv file
* Copy your Flask script
* Run it with gunicorn
When you build your image, what's the digest for agrigorev/zoomcamp-model:3.8.12-slim?

Look at the first step of your build log. It should look something like that:

Step 1/3 : FROM python:3.8.12-slim
 ---> 2e56f6b0af69

### Question 6

Let's run your docker container!

After running it, score the same customer:

url = "YOUR_URL"
customer = {"contract": "two_year", "tenure": 12, "monthlycharges": 10}
requests.post(url, json=customer).json()
What's the probability that this customer is churning?